# Labonne - Hands-On Graph Neural Networks Using Python

## Chapter 4 - Improving embeddings with biased random walks in Node2Vec

- how to improve DeepWalk/the quality of the embeddings we produced in chapter 3??
- we can do this by modifying how the random walks are generated
- we are going to see how to find the best params for a given graph, and implement **Node2Vec**
- we use Node2Vec to build a **movie recommendation system** 

## Node2Vec

- 2016
- random walks and word2vec like DeepWalk
- difference is instead of obtaining sequences of nodes with a uniform distribution, the random walks are biased

### Defining a neighborhood

- the key concept in Node2Vec is the "flexible" idea of what the neighborhood of a node is

Say you wannt to explore 3 nodes in neighborhood of a node, A.

1. you could say "get the 3 closest" **this corresponds to doing BFS**
2. or you could say "get nodes which are not adjacent to the previous node" **this is doing DFS**

These sampling strategies can be thought of as focusing on "local" vs "global" view of the graph around a node

Node2Vec says that, counterintuitively (?), the different approaches capture different but valuable representations of the network

### Introducing biases in random walks

Reminder; the idea here is that "nodes that appear together often in random walks are like words that appear together often in sentences" - under the **homophily hypothesis** they share a similar meaning, hence should share a similar representation.

In Node2Vec idea is to bias randomness of the walks to either:

1. promote nodes that are close to the previous one (similar to BFS)
2. promote nodes that are not connected to previous one (similar to DFS)

#### Theory (again a bit unclear - just code it then search explanation)

Imagine a graph where you are at current node j, there is a previous node i, and future nodes k1 and k2 (both connected to j. k1 is also connected to i, but k2 is not)

- call P_jk  the unnormalized transition prob from node j to node k
- `P_jk = a(i,k) * W_jk` were `a(i,k)` is the **search bias** between node i and node k (**this is underexplained theoretically in book** basically seems to be an idea of how easy it is to get directly to "potential next node k" from the previous node, i. W_jk is the **weight of the edge** from j to k, **again underexplained in book (we haven't mentioned weights on edges at any point so far)**

In Node2Vec the **search bias terms, a, are defined based on:**

- the distance between the 2 nodes in question and
- two more parameters, p and q, (STUPID NAMES, WHY NOT DFS_COEFF and BFS_COEFF FOR EXAMPLE???) the "return parameter" and the "in-out parameter". They are approximating the importance of DFS and BFS.

Definition of a(x,y):

```
a(x,y)   |  1/p   if dist(a,b) == 0
         |  1     if dist(a,b) == 1
         |  1/q   if dist(a,b) == 2
```

Here `dist(a,b)` is the shortest path distance between nodes a and b **NOTE: NOT STATED IN BOOK - but AFAICT, since you only compute this in the case WHERE YOU ALREADY HAVE A `i,j,k` TRIPLE OF NODES AND YOU ARE CURRENTLY AT `j`, THEN YOU KNOW THAT THE MAX DISTANCE FROM `i to k` IS ALWAYS GOING TO BE 2, SINCE AT WORST YOU CAN TAKE THE PATH `i -> j -> k` which exists by assumption**


**NOTE: AGAIN REALLY UNCLEAR IN BOOK - IN ILLUSTRATION IT SHOWS `j -> i` BEING a = 1/p BUT WHY!?!??! THE DISTANCE BETWEEN THESE 2 NODES IS 1 ??!?! NOT 0 ?!?!** It says as explantion "the walk starts at i and now arrives at j. The probability of going back to previous node i is controlled by parameter p". (So is the stuff about dist==0 above incorrect????)

Later on he says instead (p55): "1/p if neighbor is previous node, 1 if neighbor is connected to previous node, 1/q otherwise"

---

**SO BASICALLY: if DFS parameter `p` is BIG then `1/p` is SMALL so unlikely walk will return to previous node**

---

## Implementation

**NOTE: HIS EXAMPLE NOW IS ON UNWEIGHTED GRAPH so we don't even use the W defined above, just the "search bias" `a` term**



In [1]:
import networkx as nx
import random
import numpy as np

np.random.seed(0)
random.seed(0)

G = nx.erdos_renyi_graph(10, # number of nodes 
                         0.3, # connection prob / probab of an edge between any 2 nodes
                         seed=1,
                         directed=False)

**NOTE: in above I've used `a` but this is ALPHA so in below code when you see ALPHA this is the same as `a` above**

In [12]:
list(G.neighbors(0)), list(G.neighbors(4)) # ok works

([1, 4, 9], [0, 3, 5, 6, 7, 9])

In [13]:
# NOTE: recall that p and q are the "DFS" and "BFS" params respectively
def next_node(previous, current, p, q):
    # get neighbors of current node
    neighbors = list(G.neighbors(current))
    
    # calculate the appropriate alpha value for each neighbor
    # "1/p if neighbor is previous node, 1 if neighbor is connected to previous node, 1/q otherwise"
    alphas = []
    for neighbor in neighbors:
        if neighbor == previous:
            alpha = 1/p
        elif G.has_edge(neighbor, previous):
            # neighbor is directly connected to previous node
            alpha = 1
        else:
            alpha = 1/q
        alphas.append(alpha)
        
    # normalize to create transition probabilities
    probs = [alpha / sum(alphas) for alpha in alphas]
    
    # randomly select next node based on the transition probabilities, and return that node
    next_node_ = np.random.choice(neighbors, size=1, p=probs)
    
    return next_node_ # NOTE BAD VARIABLE NAME: he calls this next but it shadows Python next. I would call the function get_next_node() or something O_o
    

In [18]:
for test_num in range(10):
    display(next_node(0,4, p=1,q=1)) # ok generates random choices

array([6])

array([5])

array([9])

array([9])

array([5])

array([7])

array([6])

array([6])

array([9])

array([0])

In [21]:
xxx = str(next_node(0,4, p=1,q=1))

print(xxx)

[7]


### Update the random_walk() function, needs to include p and q, and use our updated next_node

**NOTE: CODE IN BOOK IS WRONG:**

He doesn't add `[0]` to access the content of the ndarray (of size 1 i.e. only index 0 contains the name of the next node you want)

Also for some reason he then converts the labels to str(x), but book examples show e.g. `[4,13,1,...]` the random walks as having `ints` in them. Maybe editor changed code or whatever O_o

So later you get an ndarray error

In [24]:
# update the random walk 
def random_walk(start, length, p, q):
    walk = [start]
    
    for _ in range(length):
        current = walk[-1]
        previous = walk[-2] if len(walk) > 1 else None
        next_node_ = next_node(previous, current, p, q)[0] # I ADDED [0] HERE TO GET THE ELEMENT FROM THE ndarray
        walk.append(next_node_)
        
    #return [str(x) for x in walk] # O_o this is in book, but his examples seem to want INTS
    return walk 

In [25]:
# test generate some random walks now
# ex: start at node 0, length 8
# here p=q=1 so we are doing DeepWalk

random_walk(0, 8, p=1, q=1)

[0, 1, 9, 1, 2, 1, 9, 1, 6]

Now let's bias to returing to previous node, by setting `q = 10` **RECALL, instead of the very unclear naming convention p and q, that q is BFS_COEFF and p is DFS_COEFF** so `q=10` means we are enforcing **BFS-type exploration**

In [26]:
random_walk(0, 8, p=1, q=10)

[0, 1, 9, 1, 9, 1, 9, 0, 4]

Same same for DFS exploration, setting p to 10 (**expect to explore more now rather than keep returning**):

In [27]:
random_walk(0, 8, p=10, q=1)

[0, 9, 1, 6, 5, 2, 1, 0, 4]

## Implementing Node2Vec

- use Zach Karate Club dataset as in chapter 3
- basically same as ch03 just change next_node() function and the p and q params

In [28]:
from gensim.models.word2vec import Word2Vec

# for the classifier we build with the word embeddings
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [29]:
G = nx.karate_club_graph()

In [31]:
# transform labels to ints as before

labels = []
for node in G.nodes:
    label = G.nodes[node]['club']
    labels.append(1 if label == 'Officer' else 0)

In [32]:
# generate list of random walks; 80 for each node as before
# length 10 as before
# NEW: here we use the p and q params (3 and 2 in this example)

walks = []
for node in G.nodes:
    for _ in range(80):
        walks.append(random_walk(node, 10, p=3, q=2)) # p=3, q=2

In [33]:
# create a skip-gram Word2Vec with hierarchical softmax

node2vec = Word2Vec(walks,
                   hs=1,
                   sg=1,
                   vector_size=100,
                   window=10,
                   workers=2,
                   min_count=1,
                   seed=0,
                   )

In [34]:
# train the node2vec on the random walks we generated

node2vec.train(walks,
              total_examples=node2vec.corpus_count,
              epochs=30,
              report_delay=1,
              )

(185842, 897600)

In [35]:
# --- NOW USE THESE IN A CLASSIFICATION TASK ---

# CODE IN BOOK IS WRONG, the stuff to string doesn't work - just use the ints O_o

train_mask = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]
#train_mask_str = [str(x) for x in train_mask]
test_mask = [0, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33]
#test_mask_str = [str(x) for x in test_mask]
labels = np.array(labels)

In [37]:
clf = RandomForestClassifier(random_state=0)
clf.fit(node2vec.wv[train_mask], labels[train_mask])

RandomForestClassifier(random_state=0)

In [39]:
y_pred = clf.predict(node2vec.wv[test_mask])
acc = accuracy_score(y_pred, labels[test_mask])
print(f'Node2Vec accuracy = {acc*100:.2f}%')

Node2Vec accuracy = 95.45%


In the book he shows multiple different versions with p,q varying.

Also says to repeat each experiment e.g. 100 times to get a std_dev on the accuracy and a mean value.

(Note that p=q=1 is the basic DeepWalk implementation)

**he finds that high `p` leads to better performance:** which validates the "homophily hypothesis" - knowing the graph is a social network, we anticipated that biasing towards homophily (using DFS - NOTE THAT MANY PEOPLE ASSUME BFS IS THE "homophily" IMPLEMENTATION BUT IT IS INDEED DFS that captures this property)


---

# Builing a movie recommendation system

- if we encoded movies instead of words, we can ask for movies that are similar to a given input

But how to get a graph dataset of movies where "similar movies" are connected to each other in some way??

**This bit is again unclear on first reading, but will just code it and see:** one approach is to look at ratings - we will create a graph where *movies that are liked by the same users are connected*, then use this graph to learn movie embeddings.

(The part that isn't clear is the yes/no edge question - you have 2 movies M, N. Each has a list of users who liked that movie. What, clearly explained please, is the condition for an edge between M and N then?? Do the 2 lists have to be exactly identical??? Then this will create just fully connected subgraphs of movies, indexed/labeled by their specific list of users that like all of them ??? Or are we saying that as long as 1 user is common to the 2 lists, then create an edge M to N. Or some intermediate number of "Jaccard similarity" between the 2 user lists??)

---

We use MovieLens[2] dataset: 100,000+ ratings of ~10,000 movies by 600+ users

In [40]:
# get MovieLens[2] dataset

from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

url = 'https://files.grouplens.org/datasets/movielens/ml-100k.zip'

with urlopen(url) as zurl:
    with ZipFile(BytesIO(zurl.read())) as zfile:
        zfile.extractall('.')

ratings.csv has all the users' ratings, movies.csv has movie_ids -> titles

In [41]:
import pandas as pd

ratings = pd.read_csv('ml-100k/u.data', sep='\t',names=['user_id', 'movie_id', 'rating', 'unix_timestamp'])

ratings

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [43]:
movies = pd.read_csv('ml-100k/u.item', sep='|', usecols=range(2), names=['movie_id', 'title'], encoding='latin-1')

movies

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


## NOTE: READING AHEAD

I had to read ahead because it was so unclear what is actually going to be measured:

We are going to:

- for each user, make a list of the movies he likes
- for each PAIR in this list, increment a global dict that COUNTS how many times a MOVIE PAIR occurs: this count is maintained globally across ALL THE USERS
- so basically, if there are 3 people overall who like both "movie XA" and "movie YF" then the pair (XA,YF) has value 3


**NOTE: keys are wrong in book, should be `user_id` , `movie_id` etc. maybe older version of dataset was used**

In [53]:
from collections import defaultdict

# THIS IS THE "movie pairs" dict
pairs = defaultdict(int)

# group the dataframe by users, to get each users LIKED MOVIES
for individual_user_info in ratings.groupby("user_id"):
    # individual_user_info is a 2-ple (number_here, the_info_of_dataframe_here) where number_here seems to be an index/enumerator
    # so only need the [1] index of this 2-ple to get the groupby dataframe for this particular user
    user_movies = list(individual_user_info[1]["movie_id"])
    
    # increment the counter for the pairs that are seen together in this particuar users' list
    for first_movie in range(len(user_movies)):
        for second_movie in range(first_movie+1, len(user_movies)):
            current_movie_pair = (first_movie, second_movie)
            pairs[current_movie_pair] += 1
    

In [55]:
G = nx.Graph()

### WEIGHTED GRAPH

So finally it's explained what the edge is going to be - it is **NOT** just a yes/no edge, as I as wondering earlier

**WE ARE CREATING A WEIGHTED GRAPH (first time this is mentioned in the book!!!!!!!!)**

We connect 2 movies with a weighted edge, where the weight is the co-occurence count from the above `pairs` dict

**We impose a threshold of > 20 score/co-occurence count to include a weighted edge - otherwise the graph would be huge and connections would be less meaningful** (NOTE: I'm guessing this threshold is a hyperparameter you can play with later)

In [56]:
movie_cooccurence_threshold = 20

for pair in pairs:
    first_movie, second_movie = pair # can just unpack directly above O_o
    score = pairs[pair]
    
    if score >= movie_cooccurence_threshold:
        G.add_edge(first_movie, second_movie, weight=score) # FIRST TIME WE USE WEIGHTED GRAPHS
        
    

Note that since update to dataset, we have much larger graph that he does in book (he says it has 410 nodes but 14_936 edges - I find 75_078 edges O_o)

Might need to increase threshold for the exercise to not time out

In [60]:
len(G.nodes), len(G.edges) 

(388, 75078)

**We could use our node2vec implementation - but let's try the Python library now:**

In [61]:
!pip install node2vec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
osmnx 1.9.2 requires shapely>=2.0, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.


In [62]:
from node2vec import Node2Vec

Create Node2Vec instance that will generate biased random walks, based on p and q params:

In [63]:
node2vec = Node2Vec(G,
                    dimensions=64, # IM GUESSING THIS IS THE SAME AS vector_size IN WORD2VEC API - WHY CAN'T PEOPLE STANDARDIZE THINGS EVEN IN SIMPLE LIBRARIES!??!?!
                    walk_length=20,
                    num_walks=200,
                    p=2,
                    q=1,
                    workers=1)

Computing transition probabilities:   0%|          | 0/388 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 200/200 [02:32<00:00,  1.31it/s]


Train a model on these biased random walks

**Window of 10 means 5 nodes before and 5 nodes after**

In [64]:
model = node2vec.fit(window=10,
                     min_count=1,
                     batch_words=4)

Now model is trained so use it same way as previous objects we built ourselves.

We create a function to recommend similar movies based on an input title:

`recommend()` starts by converting movie title to a movie ID we can use to query the model

In [71]:
movie = "GoldenEye (1995)"

str(movies[movies.title == movie].movie_id.values[0])

'2'

In [94]:
def recommend(movie):
    movie_id = str(movies[movies.title == movie].movie_id.values[0])
    
    # loop through 5 most similar word (i.e. NODE) vectors
    # convert these back into movie titles and output them:
    # === ALSO OUTPUT THEIR SIMILARITY SCORE ===
    for recommended_movie_id in model.wv.most_similar(movie_id)[:5]:
        #print(type(recommended_movie_id[0]), recommended_movie_id)
        
        id2title = movies[movies.movie_id == int(recommended_movie_id[0])].title.values[0]

        print(f"{id2title} : {recommended_movie_id[1]}")

In [96]:
recommend('Star Wars (1977)') # NOTE I DON'T GET SAME RESULTS AS HIM 

# he gets Return of the Jedi and Raiders of the lost ark which seems more plausible

Mad Love (1995) : 0.7250682711601257
Madness of King George, The (1994) : 0.7220710515975952
Toy Story (1995) : 0.7145920991897583
Belle de jour (1967) : 0.706836462020874
Outbreak (1995) : 0.6971874237060547


In [95]:
recommend('GoldenEye (1995)')

Angels and Insects (1995) : 0.6641371250152588
Dead Man Walking (1995) : 0.6608666181564331
Muppet Treasure Island (1996) : 0.6513866782188416
Usual Suspects, The (1995) : 0.6510480642318726
Dolores Claiborne (1994) : 0.6439917683601379


# End of chapter outlook

We are now going to address one overlooked issue in DeepWalk and Node2Vec - nodes don't have proper features yet

We will try to address this (? not clear yet what this means) with traditional neural networks, which cannot understand network topology though. This will lead us to study, finally, Graph Neural Networks